# CDAT Migration Regression Testing Notebook (`.png` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.


## Setup Code


In [5]:
import glob

from auxiliary_tools.cdat_regression_testing.utils import get_image_diffs

DEV_DIR = "843-migration-phase3-model-vs-model"
DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{DEV_DIR}/"
DEV_GLOB = sorted(glob.glob(DEV_PATH + "**/**/*.png"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_DIR = "main-model-vs-model"
MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{MAIN_DIR}/"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "**/**/*.png"))
MAIN_NUM_FILES = len(MAIN_GLOB)

In [6]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_dev_files = []
    missing_main_files = []

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(MAIN_PATH, DEV_PATH)

        if fp_dev not in DEV_GLOB:
            missing_dev_files.append(fp_dev)

    for fp_dev in DEV_GLOB:
        fp_main = fp_dev.replace(DEV_PATH, MAIN_PATH)

        if fp_main not in MAIN_GLOB:
            missing_main_files.append(fp_main)

    return missing_dev_files, missing_main_files

In [7]:
len(DEV_GLOB), len(MAIN_GLOB)

(300, 143)

## 1. Check for matching and equal number of files


In [8]:
_check_if_files_found()

In [9]:
missing_dev_files, missing_main_files = _check_if_missing_files()

In [10]:
missing_dev_files

[]

In [12]:
[f for f in missing_main_files if "diff" not in f]

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-200-ANN-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-200-JJA-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-500-ANN-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-500-JJA-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-850-ANN-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-850-JJA-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-200-ANN-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-200-JJA-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24

In [13]:
_check_if_matching_filecount()

OSError: Number of files do not match at DEV_PATH and MAIN_PATH (300 vs. 143).

## 2 Compare the plots between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [14]:
for main_path in MAIN_GLOB:
    dev_path = main_path.replace(MAIN_PATH, DEV_PATH)
    print("Comparing:")
    print(f"    * {main_path}")
    print(f"    * {dev_path}")

    get_image_diffs(dev_path, main_path)

Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/area_mean_time_series/FLUT/FLUT.png
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3-model-vs-model/area_mean_time_series/FLUT/FLUT.png
     * Difference path /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3-model-vs-model/area_mean_time_series/FLUT_diff/FLUT.png
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/area_mean_time_series/FSNTOA/FSNTOA.png
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3-model-vs-model/area_mean_time_series/FSNTOA/FSNTOA.png
     * Difference path /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3-model-vs-model/area_mean_time_series/FSNTOA_diff/FSNTOA.png
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/area_mean_time_series/LHFLX/LHFLX.png
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3-model-vs

### Results

All the plots are virtually identical. There looks like one red dot that is different, which creates a diff plot.
